In [ ]:
import itertools
import pprint
import queue
import copy

In [ ]:
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9]

class Kakuro:

    def __init__(self, rows, columns):
        # Crear matriz general
        self.matrix = [[0 for _ in range(columns)] for _ in range(rows)]
        # Crear matriz de combinaciones
        self.combinations_matrix = [[{1, 2, 3, 4, 5, 6, 7, 8, 9} for _ in range(len(self.matrix[0]))] for _ in range(len(self.matrix))]
    # end def

    def add_black_cell(self,i,j, value_column, value_row):
        self.matrix[i][j] = (value_column,value_row)
    # end def

    def play(self,i,j,value):
        if self.matrix[i][j] in numbers or self.matrix[i][j]==0:
            print("i: ",i)
            print("j: ",j)
            print("value: ",value)
            self.matrix[i][j] = value
        # end if
    # end def

    # Función tomada de: ChatGPT para facilitar la impresión y verificación de la matriz
    def print_matrix(self):
        for row in self.matrix:
            row_str = ""
            for cell in row:
                if isinstance(cell, tuple):
                    value_column, value_row = cell
                    row_str += f"({value_column},{value_row})\t"
                else:
                    row_str += f"{cell}\t"
                # end if
            # end for
            print(row_str)
        # end for
    # end def

    def verify(self):
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):
                cell = self.matrix[i][j]
                if not isinstance(cell, tuple) and cell == 0 :
                    return False
                # end if
            # end for
        # end for
        return True
    # end def

    def calculate_plays(self):

        # Recorre la matriz de probabilidaddes
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):

                # Verifica que no sea una celda negra ni una celda para colocar
                cell = self.matrix[i][j]
                if isinstance(cell, tuple):
                    self.calculate_posible_values(i,j)
                # end if

            # end for
        # end for

        # Genera la cola de prioridad a partir de las combinaciones dadas
        return self.create_priority_queue()
    # end def

    def calculate_posible_values(self,i,j):

        cell = self.matrix[i][j]

        # Evaluación de casos de columnas
        if cell[0] != -1:
            z = i + 1
            count = 0
            remain = cell[0]
            numbers_used = set()

            while z < len(self.matrix) and (self.matrix[z][j] == 0 or self.matrix[z][j] in numbers):
                remain = remain - self.matrix[z][j]
                if(self.matrix[z][j]==0):
                    count = count + 1
                else:
                    numbers_used.add(self.matrix[z][j])
                # end if
                z = z + 1
            # end while

            possible_numbers = self.calculate_combinations(count,remain,numbers_used)

            for m in range(i+1,z):
                if(self.matrix[m][j]==0):
                    current_set = set(self.combinations_matrix[m][j])
                    self.combinations_matrix[m][j] = current_set.intersection(possible_numbers)
                # end if
            # end for
        # end if

        # Evaluación de casos de filas
        if cell[1] != -1:
            z = j + 1
            count = 0
            remain = cell[1]
            numbers_used = set()

            while z < len(self.matrix[0]) and (self.matrix[i][z] == 0 or self.matrix[i][z] in numbers):
                remain = remain - self.matrix[i][z]
                if(self.matrix[i][z] == 0):
                    count = count + 1
                else:
                    numbers_used.add(self.matrix[i][z])
                # end if
                z = z + 1
            # end while

            possible_numbers = self.calculate_combinations(count,remain,numbers_used)

            for m in range(j+1,z):
                if(self.matrix[i][m] == 0):
                    current_set = set(self.combinations_matrix[i][m])
                    self.combinations_matrix[i][m] = current_set.intersection(possible_numbers)
                # end if
            # end for
        # end if

    # end def

    def calculate_combinations(self, count, number, numbers_used):
        # Generar combinaciones de números distintos
        combinations_result = list(itertools.combinations(numbers, r=count))

        # Filtrar las combinaciones para incluir solo aquellas cuya suma es igual al número deseado
        valid_combinations = [comb for comb in combinations_result if sum(comb) == number]

        # Retornar los números distintos de las combinaciones válidas que no están en numbers_used
        valid_numbers = set(num for comb in valid_combinations for num in comb if num not in numbers_used)

        return valid_numbers
    #end def

    def create_priority_queue(self):
        pq = queue.PriorityQueue()

        # Iterar sobre la matriz de combinaciones y crear la pq a partir de las celdas que poseen
        # menos jugadas disponibles
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):
                if self.matrix[i][j] == 0:
                    pq.put((len(self.combinations_matrix[i][j]), (i, j, self.combinations_matrix[i][j])))
                # end if
            # end for
        # end for

        return pq
    # end def

    def solve_kakuro_main(self):
        self.matrix = self.solve_kakuro()
        print("La solución es:")
        self.print_matrix()

    def solve_kakuro(self):

        # Si la matriz ya se encuentra solucionada, retorna la matriz
        if self.verify():
          return self.matrix

        # En caso contrario se realiza la impresión de valores
        print("----")
        self.print_matrix()
        print("----")

        # Heurística: Obtener cada una de las posibles jugadas de las celdas y utilizar aquella
        # con menos cantidad de jugadas posibles (Utilizando priority queue)
        pq = self.calculate_plays()
        actual_value = pq.get()
        print("El valor actual es: ", actual_value)

        # Hacer una copia profunda del conjunto de jugadas
        # (Utilizado en Python para no perder información en BackTracking)
        plays = copy.deepcopy(actual_value[1][2])

        while len(plays) > 0:

            # Realizar cambios
            matrix_copy = copy.deepcopy(self.matrix)
            combinations_copy = copy.deepcopy(self.combinations_matrix)
            self.play(actual_value[1][0], actual_value[1][1], plays.pop())

            # Recursión búsqueda de una solución válida dadas las posibles jugadas
            solution = self.solve_kakuro()

            # Si la solución existe, la retorna
            if solution is not None:
                return solution

            # Restaurar matrices si la solución particular no es válida
            self.matrix = matrix_copy
            self.combinations_matrix = combinations_copy

        # Si no existe una solución válida general, retorna None
        return None

    # end def


In [ ]:
def load_file(name_file):
    with open(name_file, 'r') as file:
        columns, rows = map(int, file.readline().split())
        kakuro = Kakuro(rows, columns)

        for line in file:
            values = list(map(int, line.strip().split()))
            i, j, value_column, value_row = values
            kakuro.add_black_cell(i, j, value_column, value_row)

    return kakuro

In [ ]:
kakuro = (load_file("kakuro_05.txt"))
kakuro.solve_kakuro_main()

----
(-1,-1)	(-1,-1)	(10,-1)	(11,-1)	(-1,-1)	
(-1,-1)	(4,5)	0	0	(16,-1)	
(-1,18)	0	0	0	0	
(-1,14)	0	0	0	0	
(-1,-1)	(-1,4)	0	0	(-1,-1)	
----
El valor actual es:  (1, (3, 4, {7}))
i:  3
j:  4
value:  7
----
(-1,-1)	(-1,-1)	(10,-1)	(11,-1)	(-1,-1)	
(-1,-1)	(4,5)	0	0	(16,-1)	
(-1,18)	0	0	0	0	
(-1,14)	0	0	0	7	
(-1,-1)	(-1,4)	0	0	(-1,-1)	
----
El valor actual es:  (1, (2, 4, {9}))
i:  2
j:  4
value:  9
----
(-1,-1)	(-1,-1)	(10,-1)	(11,-1)	(-1,-1)	
(-1,-1)	(4,5)	0	0	(16,-1)	
(-1,18)	0	0	0	9	
(-1,14)	0	0	0	7	
(-1,-1)	(-1,4)	0	0	(-1,-1)	
----
El valor actual es:  (1, (3, 1, {1}))
i:  3
j:  1
value:  1
----
(-1,-1)	(-1,-1)	(10,-1)	(11,-1)	(-1,-1)	
(-1,-1)	(4,5)	0	0	(16,-1)	
(-1,18)	0	0	0	9	
(-1,14)	1	0	0	7	
(-1,-1)	(-1,4)	0	0	(-1,-1)	
----
El valor actual es:  (1, (2, 1, {3}))
i:  2
j:  1
value:  3
----
(-1,-1)	(-1,-1)	(10,-1)	(11,-1)	(-1,-1)	
(-1,-1)	(4,5)	0	0	(16,-1)	
(-1,18)	3	0	0	9	
(-1,14)	1	0	0	7	
(-1,-1)	(-1,4)	0	0	(-1,-1)	
----
El valor actual es:  (1, (3, 3, {2}))
i:  3
j:  3
value:  2
